In [2]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving 50_Startups.csv to 50_Startups.csv
User uploaded file "50_Startups.csv" with length 2436 bytes


In [1]:
from sklearn.utils import shuffle
import pandas as pd
import numpy as np

In [4]:
dataset = pd.read_csv('50_Startups.csv')
dataset.head()

,R&D Spend,Administration,Marketing Spend,State,Profit
0,165349.20,136897.80,471784.10,New York,192261.83
1,162597.70,151377.59,443898.53,California,191792.06
2,153441.51,101145.55,407934.54,Florida,191050.39
3,144372.41,118671.85,383199.62,New York,182901.99
4,142107.34,91391.77,366168.42,Florida,166187.94


In [43]:
# 获取x值和y值
X_ds = dataset.iloc[:,:-1].values
y_ds = dataset.iloc[:,-1].values.reshape((-1,1))

In [44]:
# 给离散值编码
label_encoder = LabelEncoder()
X_ds[:,3] = label_encoder.fit_transform(X_ds[:,3])
X_ds[:,3]

array([2, 0, 1, 2, 1, 2, 0, 1, 2, 0, 1, 0, 1, 0, 1, 2, 0, 2, 1, 2, 0, 2,
       1, 1, 2, 0, 1, 2, 1, 2, 1, 2, 0, 1, 0, 2, 1, 0, 2, 0, 0, 1, 0, 2,
       0, 2, 1, 0, 2, 0], dtype=object)

In [36]:
# 归一化
X_ds = np.array(X_ds, dtype=np.float32)
men = np.mean(X_ds, axis=0)
st = np.std(X_ds, axis=0)
X_ds = (X_ds - men) / st

In [48]:
X_ds = np.array(X_ds, dtype=np.float64)

In [38]:
class LinearReg:
    """
    线性回归
    """

    def __init__(self, data_x, data_y):
        """
        :param data_x: 数据集X部分（归一化后的）
        :param data_y: 数据集y部分
        """
        self.X = np.hstack((data_x, np.ones((data_x.shape[0], 1))))
        self.X = self.X.astype(np.float32)
        self.y = data_y
        self.w = np.random.rand(self.X.shape[1], 1)  # 回归参数

    def get_separated_data(self, ratio, seed):
        """
        划分训练集和测试集
        :param ratio: 训练集占比（0-1之间的数）
        :param seed: 随机数种子
        :return: 训练集和测试集数据
        """
        X, y = shuffle(self.X, self.y, random_state=seed)
        offset = int(self.X.shape[0] * ratio)
        X_train, y_train = X[:offset], y[:offset]
        X_test, y_test = X[offset:], y[offset:]
        return X_train, y_train, X_test, y_test

    def get_loss(self, X, y):
        """
        计算预测值（m维向量），数据集平均损失，参数梯度
        :param X: 数据集x部分
        :param y: 数据集y部分
        :return: 预测值（m维向量），数据集平均损失，参数梯度（n维向量）
        """
        m = X.shape[0]
        y_hat = np.dot(X, self.w)
        loss = np.sum((y_hat - y) ** 2) / m
        dw = np.dot(X.T, (y_hat - y)) / m
        return y_hat, loss, dw

    def model_train(self, X_train, y_train, learning_rate, epochs):
        for i in range(1, epochs):
            y_hat, loss, dw = self.get_loss(X_train, y_train)
            self.w += -learning_rate * dw
            if i % 500000 == 0:
                print('epoch %d loss %f' % (i, loss))

    def data_predict(self, x):
        print(x.shape, self.w.shape)
        return np.dot(x, self.w)

In [49]:
lr_model = LinearReg(X_ds, y_ds)
X_train, y_train, X_test, y_test = lr_model.get_separated_data(0.9, 0)

In [2]:
learning_rate = 0.5
epochs = 10000000
lr_model.model_train(X_train, y_train, learning_rate, epochs)

In [42]:
def compute_para_direct(X, y):
  a =  np.linalg.inv(np.dot(X.T, X))
  b = np.dot(X.T, y)
  w = np.dot(a,b)
  m = X.shape[0]
  y_hat = np.dot(X, w)
  loss = np.sum((y_hat - y) ** 2) / m
  return w, loss
w,loss = compute_para_direct(X_train, y_train)
print(loss)

83000465.71897252


In [63]:
idx = np.random.randint(0, X_test.shape[0])
y = y_test[idx]
y_hat = np.dot(X_test[idx], lr_model.w)
print(y, y_hat)

[209.] [154.04741475]


In [64]:
y_hat = np.dot(X_test[idx], w)
print(y, y_hat)

[209.] [154.04732343]
